In [23]:
import cv2
import numpy as np
from pathlib import Path

# Carpetas de entrada y salida
input_folder = Path("alas_png")
output_folder = Path("alas_neon")
output_folder.mkdir(exist_ok=True)

# Procesar todas las imágenes PNG de la carpeta
for img_path in input_folder.glob("*.png"):
    # --- Cargar imagen ---
    img = cv2.imread(str(img_path))
    if img is None:
        continue

    # Escala de grises + suavizado
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)

    # Segmentación binaria (Otsu)
    _, mask = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    mask = cv2.bitwise_not(mask)

    # --- Extraer ala sobre fondo negro ---
    ala_segmentada = cv2.bitwise_and(img, img, mask=mask)

    # --- Aplicar efecto neón ---
    ala_gray = cv2.cvtColor(ala_segmentada, cv2.COLOR_BGR2GRAY)
    neon = cv2.applyColorMap(ala_gray, cv2.COLORMAP_PLASMA)
    ala_neon = cv2.bitwise_and(neon, neon, mask=mask)

    # --- Fondo negro ---
    h, w, _ = img.shape
    fondo_negro = np.zeros((h, w, 3), dtype=np.uint8)

    # --- Estrellas de colores ---
    stars = fondo_negro.copy()
    mask_stars = np.random.rand(h, w) < 0.0015
    rand_colors = np.random.randint(150, 256, (np.sum(mask_stars), 3))
    stars[mask_stars] = rand_colors

    # --- Combinar ala + estrellas ---
    resultado = cv2.addWeighted(fondo_negro, 1.0, stars, 1.0, 0)
    resultado = cv2.addWeighted(resultado, 1.0, ala_neon, 1.2, 0)

    # --- Guardar en carpeta de salida ---
    out_path = output_folder / img_path.name
    cv2.imwrite(str(out_path), resultado)

print("Procesamiento completado. Imágenes guardadas en 'alas_neon'")

Procesamiento completado. Imágenes guardadas en 'alas_neon'
